# Mayer & Nelson (2020) Phonotactic learning with neural language models

In [1]:
import random
import torch
import torch.nn as nn
import sys
import numpy as np
import time
from scipy.stats import norm

# Part 1: Getting data

The first step is to get the data into Python.

The raw data looks like this:

    a a k k o n e n
    a a k k o s e l l i n e n
    a a k k o s e l l i s e s t i
    a a k k o s e l l i s u u s
    a a k k o s i t t a i n
    a a k k o s j ae r j e s t y s
    a a k k o s n u m e e r i n e n
    a a k k o s t a a

Each line is a word and each phoneme is separated by spaces.

The function `get_corpus_data()` takes an input file, and turns it into a list of lists, with each item of the larger list being a list of phonemes. Note that `str.rstrip()` removes **trailing** space characters, i.e at the end, but not those characters at the beginning. So the line breaks at the end of each line are removed.

In [45]:
def get_corpus_data(filename):
    """Reads input file and coverts it to list of lists, adding word boundary 
    markers <s> (start) and <e> (end).
    """
    raw_data = []
    file = open(filename, 'r', encoding="utf8")
    for line in file:
        line = line.rstrip()
        line = ['<s>'] + line.split(' ') + ['<e>']
        raw_data.append(line)
    return raw_data

In [3]:
raw_data_finnish = get_corpus_data("../sample_data/corpora/finnish_training.txt")

The next function, `process_data()`, takes the data we just got, and turns it into a dataset through the following steps:
1. The data is shuffled
2. Each word is padded to the maximum length with the padding character `<p>` so that each data point has identical length
3. Gets the inventory, i.e. every possible phone in the data, with `<p>` at index 0
4. Assigns indices to the phones, and provides dictionaries to convert from one to the other
5. Creates the training and dev sets for PyTorch using `torch.LongTensor()`, a tensor data type that stores tensors

In [4]:
def process_data(string_training_data, dev=True, training_split=60):
    """

    Args:
        string_training_data (list(list(chr))): A list where each item
                                                is a list of phones
                                                from a word.
        dev (bool, optional): Whether we want to split out a dev set.
                              Defaults to True.
        training_split (int, optional): Percentage of data used for
                                        training. Defaults to 60.

    Returns:
        tuple: A tuple containing five items: the 'vocabulary' (i.e.
               phone inventory), a dictionary for looking up indices
               from phones, a dictionary for looking up phones from
               indices, the training set, and the development set.
    """

    random.shuffle(string_training_data)
    # all data points need to be padded to the maximum length
    max_chars = max([len(x) for x in string_training_data])
    string_training_data = [
        sequence + ['<p>'] * (max_chars - len(sequence)) 
        for sequence in string_training_data]
    # get the inventory and build both directions of dicts  
    # this will store the set of possible phones
    inventory = list(set(phone for word in string_training_data for phone in word))
    inventory = ['<p>'] + [x for x in inventory if x != '<p>'] #ensure that the padding symbol is at index 0

    # dictionaries for looking up the index of a phone and vice versa
    phone2ix = {p: ix for (ix, p) in enumerate(inventory)}
    ix2phone = {ix: p for (ix, p) in enumerate(inventory)}

    as_ixs = [
        torch.LongTensor([phone2ix[p] for p in sequence])
        for sequence in string_training_data
      ]

    if not dev:
        training_data = torch.stack(as_ixs, 0)
        # simpler make a meaningless tiny dev than to have a different eval 
        # training method that doesn't compute Dev perplexity
        dev = torch.stack(as_ixs[-10:], 0)
    else:
        split = int(len(as_ixs) * (training_split/100))
        training_data = torch.stack(as_ixs[:split], 0)
        dev = torch.stack(as_ixs[split:], 0)

    return (inventory, phone2ix, ix2phone, training_data, dev)

In [5]:
inventory, phone2ix, ix2phone, training, dev = process_data(
        raw_data_finnish, dev=True, training_split=60
    )
inventory_size = len(inventory)

In [30]:
print(training)
print(training.size())
print(dev)
print(dev.size())

tensor([[ 5,  2, 16,  ...,  0,  0,  0],
        [ 5, 12, 19,  ...,  0,  0,  0],
        [ 5,  4, 23,  ...,  0,  0,  0],
        ...,
        [ 5,  4, 23,  ...,  0,  0,  0],
        [ 5, 16,  2,  ...,  0,  0,  0],
        [ 5,  4, 13,  ...,  0,  0,  0]])
torch.Size([56292, 32])
tensor([[ 5, 10, 22,  ...,  0,  0,  0],
        [ 5, 18,  2,  ...,  0,  0,  0],
        [ 5,  2, 18,  ...,  0,  0,  0],
        ...,
        [ 5, 15, 22,  ...,  0,  0,  0],
        [ 5,  2, 13,  ...,  0,  0,  0],
        [ 5, 21, 12,  ...,  0,  0,  0]])
torch.Size([37529, 32])
torch.Size([10, 32])
tensor([ 2, 16, 18, 17, 10, 10,  7,  8, 22, 21, 21,  7, 25,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 12, 19,
         3, 23, 17,  2, 10, 22, 10, 23,  8, 23, 19, 10, 25,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  4, 23, 21, 21,
         7, 10, 10, 23, 21, 11,  2, 22, 19, 22, 25,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 23,  6, 22, 19, 15, 23,
   

# Part 2: Defining the model

Here we define the model structure that we'll use:
- The `Emb_RNNLM` class inherits from `nn.Module`, which is the most basic neural network class in Python
- The constructor function of `Emb_RNNLM()` takes a dictionary called `params` that contains the following info:
    - `inv_size`: Size of the phone inventory, i.e. vocabulary
    - `d_emb`: Number of dimensions in the embedding
    - `n_layers`: Number of layers in the model
    - `d_hid`: Number of dimensions in hidden layer
- The embedding layer and the recurrent layer are defined with respect to these parameters:
    - The projection layer (`nn.Embedding`)'s constructor function takes the vocabulary size and the dimensionality of the embedding
      as parameters.
      - Question: What are the dimensions of its input and output? What is the activation function?
    - The recurrent layer (`nn.RNN`) implements a simple recurrent network (SRN), a.k.a. an Elman network
      - By default, tanh is used as the activation function
        - Questions: What are the input and output dimensionality?
      - Multiple hidden layers are possible, but for this paper (and this week) we'll just do one
      - `batch_first=True` tells PyTorch that the input passed to this layer will consist of the embeddings first, then the sequence lengths.
    - The output layer (`nn.Linear`) outputs a score for each phone, the softmax of which is the probability of the next phoneme
      - If the projection and output layers have the same dimension, then the model will make their weights identical.
        - Question: How does this affect model variance?

In [6]:
class Emb_RNNLM(nn.Module):
    def __init__(self, params):
        super(Emb_RNNLM, self).__init__()
        self.vocab_size = params['inv_size']
        self.d_emb = params['d_emb']
        self.n_layers = params['num_layers']
        self.d_hid = params['d_hid']
        self.embeddings = nn.Embedding(self.vocab_size, self.d_emb)
        
        # input to recurrent layer, default nonlinearity is tanh
        self.i2R = nn.RNN(
            self.d_emb, self.d_hid, batch_first=True, num_layers=self.n_layers
        )
        # recurrent to output layer
        self.R2o = nn.Linear(self.d_hid, self.vocab_size)
        if params['tied']:
            if self.d_emb == self.d_hid:
                self.R2o.weight = self.embeddings.weight
            else:
                print("Dimensions don't support tied embeddings")

    def forward(self, batch):
        batches, seq_len = batch.size()
        embs = self.embeddings(batch)
        output, hidden = self.i2R(embs)
        outputs = self.R2o(output)
        return outputs

In [7]:
rnn_params = {}
rnn_params['d_emb'] = 24
rnn_params['d_hid'] = 64
rnn_params['num_layers'] = 1
rnn_params['batch_size'] = 64
rnn_params['learning_rate'] = .005
rnn_params['epochs'] = 10
rnn_params['tied'] = True
rnn_params['inv_size'] = inventory_size
RNN = Emb_RNNLM(rnn_params)

Dimensions don't support tied embeddings


In [31]:
print(RNN(training[1:10]))

tensor([[[ 0.2000, -0.0308,  0.0767,  ..., -0.0590,  0.2589,  0.1230],
         [ 0.2363,  0.1325,  0.1683,  ...,  0.0086, -0.1846, -0.0095],
         [ 0.1430,  0.3388, -0.1756,  ...,  0.0010, -0.1351, -0.0618],
         ...,
         [ 0.1276, -0.4260, -0.0736,  ..., -0.1291, -0.3733,  0.4396],
         [ 0.1276, -0.4260, -0.0736,  ..., -0.1291, -0.3733,  0.4396],
         [ 0.1276, -0.4260, -0.0736,  ..., -0.1291, -0.3733,  0.4396]],

        [[ 0.2000, -0.0308,  0.0767,  ..., -0.0590,  0.2589,  0.1230],
         [ 0.1006,  0.3132, -0.0226,  ..., -0.3514, -0.1742, -0.0527],
         [ 0.4475,  0.2166,  0.1853,  ..., -0.0256,  0.1947, -0.3491],
         ...,
         [ 0.1276, -0.4260, -0.0736,  ..., -0.1291, -0.3733,  0.4396],
         [ 0.1276, -0.4260, -0.0736,  ..., -0.1291, -0.3733,  0.4396],
         [ 0.1276, -0.4260, -0.0736,  ..., -0.1291, -0.3733,  0.4396]],

        [[ 0.2000, -0.0308,  0.0767,  ..., -0.0590,  0.2589,  0.1230],
         [ 0.3557,  0.3604,  0.2864,  ..., -0

# Part 3: Model training

Here are the details for the function for model training:
- The cross entropy loss is equivalent to the negative log-likelihood loss,
- The Adams optimiser is used for performing gradient descent. It is a slightly
  modified version of gradient descent, which fiddles around with the learning rate
  and adds a feature called momentum to make the model learn faster.
- As with Assignment 1, the data is divided into a bunch of batches controlled by 
  batch size. The `batches` variable stores the first and last position of within
  the training set of each batch.
- Within each epoch:
  - Each epoch shuffles the order of the batches
  - Within each batch:
    - Predictions are calculated:
      - `preds` houses the predicted scores (not probabilities!)
      - `targets` houses the predictions
      - `criterion` calculates the loss, taking `preds` and `targets` as input
    - A backward pass is performed
    - Weights are updated `Optimizer.step()`
    - Gradients are cleared
    - Loss for the epoch is updated
  - Perplexity is calculated at the end of each epoch
  - If, at the end of the epoch, dev perplexity hasn't moved by more than .01,
    then we end training early.

In [32]:
def train_lm(dataset, dev, params, net):
    criterion = nn.CrossEntropyLoss(ignore_index=0)
    optimizer = torch.optim.Adam(net.parameters(), lr=params['learning_rate'])
    (num_examples, seq_len) = dataset.size()    
    batches = [
        (start, start + params['batch_size']) 
        for start in range(0, num_examples, params['batch_size'])
    ]
    
    prev_perplexity = 1e10
    for epoch in range(params['epochs']):
        ep_loss = 0.
        start_time = time.time()
        random.shuffle(batches)
        
        for b_idx, (start, end) in enumerate(batches):
            batch = dataset[start:end]
            preds = net(batch)
            preds = preds[:, :-1, :].contiguous().view(-1, net.vocab_size)
            targets = batch[:, 1:].contiguous().view(-1)
            loss = criterion(preds, targets)
            
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            ep_loss += loss.detach()
        dev_perplexity = compute_perplexity(dev, net)

        print('epoch: %d, loss: %0.2f, time: %0.2f sec, dev perplexity: %0.2f' %
              (epoch, ep_loss, time.time()-start_time, dev_perplexity))
        # stop early criterion, increasing perplexity on dev 
        if dev_perplexity - prev_perplexity > 0.01:
            print('Stop early reached')
            break

The perplexity is not automatically given, so we do need to write a function for it ourselves. Fortunately, there's an easy way to calculate it from the loss!

$$
\begin{align*}
\rho &= \exp \left(-\frac{1}{n_{tok}} \sum_{i = 1}^{n_{tok}} \log(\Pr(\text{phoneme}_i|\text{prev tokens})) \right)\\
&= \exp \left(\frac{1}{n_{tok}} L(\text{all tokens}) \right)
\end{align*}$$

(Incidentally, there is some repeated code between this function and the previous; you could potentially think of a way to make a third function to avoid this repetition!)

In [33]:
def compute_perplexity(dataset, net, bsz=64):
    criterion = nn.CrossEntropyLoss(ignore_index=0, reduction='sum')
    num_examples, seq_len = dataset.size()
    
    batches = [(start, start + bsz) for start in\
               range(0, num_examples, bsz)]
    
    total_unmasked_tokens = 0.
    nll = 0.
    for b_idx, (start, end) in enumerate(batches):
        batch = dataset[start:end]
        ut = torch.nonzero(batch).size(0)
        preds = net(batch)
        targets = batch[:, 1:].contiguous().view(-1)
        preds = preds[:, :-1, :].contiguous().view(-1, net.vocab_size)
        loss = criterion(preds, targets)
        nll += loss.detach()
        total_unmasked_tokens += ut

    perplexity = torch.exp(nll / total_unmasked_tokens).cpu()
    return perplexity.data


In [34]:
train_lm(training, dev, rnn_params, RNN)

epoch: 0, loss: 1943.01, time: 6.54 sec, dev perplexity: 6.99
epoch: 1, loss: 1835.14, time: 6.28 sec, dev perplexity: 6.76
epoch: 2, loss: 1808.34, time: 6.04 sec, dev perplexity: 6.65
epoch: 3, loss: 1794.52, time: 6.06 sec, dev perplexity: 6.55
epoch: 4, loss: 1785.98, time: 6.52 sec, dev perplexity: 6.53
epoch: 5, loss: 1780.50, time: 6.83 sec, dev perplexity: 6.48
epoch: 6, loss: 1776.29, time: 6.97 sec, dev perplexity: 6.44
epoch: 7, loss: 1773.24, time: 7.47 sec, dev perplexity: 6.48
epoch: 8, loss: 1770.87, time: 9.38 sec, dev perplexity: 6.44
epoch: 9, loss: 1768.70, time: 13.14 sec, dev perplexity: 6.42


# Part 4: Model evaluation

And finally, we get to model evaluation. The idea is to compute perplexity using the training set.

Because we want to compute perplexity for a single word each time, instead of an entire bunch of words, we only need to input a tensor representing a single word. `torch.unsqueeze()` adds an additional dimension so that the input is of the correct shape for `compute_perplexity()`, which assumes there are multiple words.

(Again, there is some repetition in code with some functions above - think about how to get rid of it!)

In [38]:
def get_probs(input_file, model, phone2ix, out_filename):
    inp_file = open(input_file, 'r', encoding='UTF-8')
    out_file = open(out_filename,'w', encoding='UTF-8')
    data_tens = []
    as_strings = []
    for line in inp_file:
        line = line.rstrip()
        as_strings.append(line.replace(' ',''))
        line = line.split(' ')
        line = ['<s>'] + line + ['<e>']
        line_as_tensor = torch.LongTensor([phone2ix[p] for p in line])
        data_tens.append(line_as_tensor)

    num_points = len(data_tens)

    for i,word in enumerate(data_tens):
        curr_string = as_strings[i]
        out_file.write(curr_string + '\t' + str(compute_perplexity(word.unsqueeze(0), model).numpy()) + '\n')
    
    inp_file.close()
    out_file.close()

Note that `RNN.eval()` tells PyTorch that we're going to use the data to be input into the neural network for evaluation only. (This doesn't have huge impacts on this particular case, but if your model contains things like batch normalisation or dropout, which we haven't discussed, this is an essential step.)

In [39]:
RNN.eval()
get_probs("../sample_data/test_data/finnish_test.txt",
    RNN, phone2ix,
    "../output/finnish.csv")


Let's look at some of the results. What do you notice?

    poemivo		15.390921
    toemagu		20.85924
    rytoky		19.299955
    jaebejaeli	27.688951
    

    rasiketo	7.663081
    hahohasu	12.35365
    nypimide	8.837763
    rojapotto	8.841386
    kentittoe	9.206199
    helesa		9.307686

# The other two case studies

Let's create a new function that includes everything we did above, but extensible to the other two case studies:

In [43]:
def run_full_pipeline(train_path, test_path, output_path):
    #Prepare data
    raw_data = get_corpus_data(train_path)
    inventory, phone2ix, ix2phone, training, dev = process_data(
        raw_data, dev=True, training_split=60
    )
    inventory_size = len(inventory)

    #Set hyperparameters and model structure
    rnn_params = {}
    rnn_params['d_emb'] = 24
    rnn_params['d_hid'] = 64
    rnn_params['num_layers'] = 1
    rnn_params['batch_size'] = 64
    rnn_params['learning_rate'] = .005
    rnn_params['epochs'] = 10
    rnn_params['tied'] = True
    rnn_params['inv_size'] = inventory_size
    RNN = Emb_RNNLM(rnn_params)

    #Train model
    train_lm(training, dev, rnn_params, RNN)

    #Evaluate model
    RNN.eval()
    get_probs(test_path,
        RNN, phone2ix,
        output_path)


Let's run the model on the Cochabamba Quechua data:

In [41]:
run_full_pipeline("../sample_data/corpora/quechua_training.txt",
    "../sample_data/test_data/quechua_test.txt",
    "../output/quechua.csv")

Dimensions don't support tied embeddings
epoch: 0, loss: 61.01, time: 0.12 sec, dev perplexity: 5.84
epoch: 1, loss: 47.82, time: 0.12 sec, dev perplexity: 5.20
epoch: 2, loss: 46.08, time: 0.12 sec, dev perplexity: 5.10
epoch: 3, loss: 45.43, time: 0.11 sec, dev perplexity: 5.04
epoch: 4, loss: 44.85, time: 0.12 sec, dev perplexity: 4.98
epoch: 5, loss: 44.46, time: 0.12 sec, dev perplexity: 4.94
epoch: 6, loss: 43.97, time: 0.11 sec, dev perplexity: 4.91
epoch: 7, loss: 43.64, time: 0.11 sec, dev perplexity: 4.90
epoch: 8, loss: 43.42, time: 0.13 sec, dev perplexity: 4.89
epoch: 9, loss: 43.15, time: 0.13 sec, dev perplexity: 4.88


What do you notice this time? (`+` indicates ejective.)

    wap+a	4.8824334
    Last+i	5.0887637
    p+unsi	6.026197
    p+awi	4.6279793

    kiLc+u	8.266438
    kajc+u	10.616985
    kap+a	7.2731633
    qajc+i	9.161771
    qajc+u	9.812296

Let's do the last English case study:

In [46]:
run_full_pipeline("../sample_data/corpora/CMU_dict_IPA.txt",
    "../sample_data/test_data/Daland_et_al_IPA.txt",
    "../output/english.csv")

Dimensions don't support tied embeddings
epoch: 0, loss: 3074.04, time: 13.04 sec, dev perplexity: 8.21
epoch: 1, loss: 2972.34, time: 13.65 sec, dev perplexity: 8.09
epoch: 2, loss: 2953.39, time: 18.52 sec, dev perplexity: 8.01
epoch: 3, loss: 2943.21, time: 17.97 sec, dev perplexity: 7.97
epoch: 4, loss: 2936.54, time: 15.58 sec, dev perplexity: 7.95
epoch: 5, loss: 2932.39, time: 14.62 sec, dev perplexity: 7.93
epoch: 6, loss: 2928.44, time: 14.61 sec, dev perplexity: 7.93
epoch: 7, loss: 2925.80, time: 14.12 sec, dev perplexity: 7.91
epoch: 8, loss: 2923.28, time: 14.00 sec, dev perplexity: 7.93
epoch: 9, loss: 2921.79, time: 15.02 sec, dev perplexity: 7.91


What do you notice this time? (`+` indicates ejective.)

Attested |Marginal |Unattested|
|---|---|---|
tw tɹ sw |gw ʃl |pw zɹ mɹ |
ʃɹ pɹ pl |vw Sw |tl dn km |
kw kɹ kl |ʃn ʃm |fn ml nl |
gr gl fɹ |vl bw |dg pk lm|
fl dɹ bɹ |dw fw |ln ɹl lt |
bl sn sm |vɹ θw |ɹn ɹd rg|

    bligɪf	13.21389
    blɛzɪg	11.064883
    dɹigɪf	13.357114
    glɛpɪd	10.587212

    gwibɪd	8.843671
    vwigɪf	24.82807
    ʃnigɪf	19.082851
    θwɛpɪd	15.313112

    dgɛpɪd	22.96906
    dnigɪf	30.790936
    lmɑtɪf	22.789053
    ɹlɛzɪg	28.106743
    ɹnɑsɪp	24.705233